In [1]:
#import modules
import pandas as pd
import numpy as np
import requests

from pandas.io import gbq
#from google.cloud import bigquery,storage
from google.oauth2 import service_account

import gspread
import json, os
from canvasapi import Canvas

from datetime import datetime

from time import time

from params import basedir, run_mode
#import plotly.express as px

In [2]:
#choose credential file paths and other possible changes:
#run_mode = 'dev' #or, 'prod', or 'mig' for when migrating the code
print(run_mode)

dev


In [3]:

if run_mode == 'dev':
    out_table = "all_courses3"
    
elif run_mode == 'prod':
    out_table = "all_courses4"

elif run_mode == 'prod_home':
    out_table = "all_courses4"

    
elif run_mode == 'mig':
    out_table = "all_courses3"



In [5]:

#get Google cloud credentials
project_id = 'canvas-portal-data-custom'
cred_file = '{}/canvas-portal-data-custom-6e244db3b826.json'.format(basedir)
data_dl = 'data'
scopes = [ "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
            "https://spreadsheets.google.com/auth/spreadsheets"]
credentials = service_account.Credentials.from_service_account_file(cred_file,)

#get Canvas credentials
#cred_file2 = '{}/instances.json'.format(basedir)
#with open(cred_file2,'r') as cred2:
#    cred_json = json.load(cred2)

from instances import canvas_api, panopto_api

canvas_API_KEY = canvas_api['ACCES_TOKEN']
canvas_API_URL = canvas_api['API_URL']#+'/accounts/1'

panopto_client_id = panopto_api['Client Id']
panopto_client_secret = panopto_api['Client Secret']

In [6]:
# Initialize a new Canvas object
canvas = Canvas(canvas_API_URL, canvas_API_KEY)
canvas.__dict__

#Initialize a new Panopto object

{'_Canvas__requester': <canvasapi.requester.Requester at 0x7fb32ee4c2b0>}

### Get courses running on Canvas -- created via migration or by LT's

In [7]:
#Get the LT list from the Excel sheet
lt_df_new_cols = ['School', 'Dept_num', 'Dept_name', 'Name', 'Email']
lt_existing_cols = ['School', 'Assigned to:', 'Department name', 'Name', 'Contact Email']
lt_df = pd.read_excel('{}/Learning_Technologists_updating.xlsx'.format(basedir), engine="openpyxl")
lt_df = lt_df[lt_existing_cols]
lt_df.columns = lt_df_new_cols
lt_df['Email'] = lt_df.Email.str.lower()

In [8]:
'''
#Read the Stellar to Canvas migration list on Google Drive, and construct a
#list of course_id s
gs_name = "Stellar to Canvas content migration request (Responses)"
#gs_name = "xyz"
gc = gspread.service_account(filename=cred_file)
sh = gc.open(gs_name).sheet1
sh_data = sh.get_all_values()
head_col = sh_data.pop(0)
stellar_df = pd.DataFrame(sh_data, columns=head_col)
stellar_df['course_id'] = stellar_df['Canvas URL to migrate to'].str.split("/").str[-1]
'''

'\n#Read the Stellar to Canvas migration list on Google Drive, and construct a\n#list of course_id s\ngs_name = "Stellar to Canvas content migration request (Responses)"\n#gs_name = "xyz"\ngc = gspread.service_account(filename=cred_file)\nsh = gc.open(gs_name).sheet1\nsh_data = sh.get_all_values()\nhead_col = sh_data.pop(0)\nstellar_df = pd.DataFrame(sh_data, columns=head_col)\nstellar_df[\'course_id\'] = stellar_df[\'Canvas URL to migrate to\'].str.split("/").str[-1]\n'

In [8]:
def get_course_info(course_id):
    '''This function gets the course information and the file/assignment update times,
    by the course_id, and returns a list of '''
    cutoff_date = datetime.strptime('2015-05-31', '%Y-%m-%d') #by this time most default content was created
    
    try:
        c1 = canvas.get_course(course_id, include='total_students')
    except:
        c1 = None
        
    try:
        enrollment_term = term_dict[c1.enrollment_term_id]
    except:
        enrollment_term = None
        
    try:
        course_dept = sub_account_dict[c1.account_id]
    except:
        course_dept = None

        
    try:
        parent_account = parent_account_dict[c1.account_id]
    except:
        parent_account = None


    try:
        course_name = c1.name
        
    except:
        course_name = None
        
        
    try:
        course_sis_id = c1.sis_course_id
        
    except:
        course_sis_id = None        

    try:
        course_state = c1.workflow_state

    except:
        course_state = None

        
    try:
        num_students = c1.total_students

    except:
        num_students = None
        
        
    #Find out if the course is public or not:
    try:    
        is_public = c1.is_public
    except:
        is_public = None


    try:
        public_syllabus = c1.public_syllabus
    except:
        public_syllabus = None


    try:
        public_syllabus_to_auth = c1.public_syllabus_to_auth
    except:
        public_syllabus_to_auth = None


    try:        
        is_public_to_auth_users = c1.is_public_to_auth_users
    except:
        is_public_to_auth_users = None

        
        
    if is_public == 1:
        course_visibility = 'public'
    elif is_public == 0 and is_public_to_auth_users == 1:
        course_visibility = 'institute'
    elif is_public == 0 and is_public_to_auth_users == 0:
        course_visibility = 'not_public_to_auth_users'
    else:
        course_visibility = 'unknown'

    #get discussion_topics, pages, quizzes, assignment_groups, modules, and module items
    try:
        list_dis_topics = c1.get_discussion_topics()
    except:
        list_dis_topics = []
        
        
    try:
        list_pages = c1.get_pages()
    except:
        list_pages = []        
        
    try:
        list_quizzes = c1.get_quizzes()
    except:
        list_quizzes = []        
        
    try:
        list_assignment_groups = c1.get_assignment_groups()
    except:
        list_assignment_groups = []        

    def count_stuff(list_items, if_published=None):
        num_stuff = 0
        for item in list_items:
            if if_published != None:
                if item.published == if_published:
                    num_stuff += 1

            elif if_published == None:
                num_stuff += 1


        return num_stuff

    num_published_dis_topics = count_stuff(list_dis_topics, if_published=True)
    num_published_pages = count_stuff(list_pages, if_published=True)
    num_published_quizzes = count_stuff(list_quizzes, if_published=True)
    num_assignment_groups = count_stuff(list_assignment_groups, if_published=None) 
    #Note: different if_published kwd

    try:
        modules_list = c1.get_modules()
    except:
        modules_list = []
    num_modules = 0
    num_module_items = 0

    for d_ in modules_list:
        num_modules += 1
        num_module_items += d_.items_count


    #get files and assignments
    try:
        files_ = c1.get_files()
    except:
        files_ = []
        
    try:
        assn_ = c1.get_assignments()
    except:
        assn_ = []
        
    #Get the file updated times
    file_utimes_all = [f_.updated_at for f_ in files_ ]        
    file_utimes = [f_.updated_at for f_ in files_ if datetime.strptime(f_.updated_at, 
                                                                    '%Y-%m-%dT%H:%M:%SZ') > cutoff_date]

    #Get the file created times
    file_ctimes_all = [f_.created_at for f_ in files_ ]
    file_ctimes = [f_.created_at for f_ in files_ if datetime.strptime(f_.created_at, 
                                                                    '%Y-%m-%dT%H:%M:%SZ') > cutoff_date]

    #Get the assignment updated times
    assn_utimes_all = [a_.updated_at for a_ in assn_ ]
    assn_utimes = [a_.updated_at for a_ in assn_ if datetime.strptime(a_.updated_at, 
                                                                    '%Y-%m-%dT%H:%M:%SZ') > cutoff_date]


    #Get the assignment created times
    assn_ctimes_all = [a_.created_at for a_ in assn_ ]
    assn_ctimes = [a_.created_at for a_ in assn_ if datetime.strptime(a_.created_at, 
                                                                    '%Y-%m-%dT%H:%M:%SZ') > cutoff_date]

    
    #Get the page updated times
    page_utimes_all = [f_.updated_at for f_ in list_pages ]        
    page_utimes = [f_.updated_at for f_ in list_pages if datetime.strptime(f_.updated_at, 
                                                                    '%Y-%m-%dT%H:%M:%SZ') > cutoff_date]

    #Get the page created times
    page_ctimes_all = [f_.created_at for f_ in list_pages ]
    page_ctimes = [f_.created_at for f_ in list_pages if datetime.strptime(f_.created_at, 
                                                                    '%Y-%m-%dT%H:%M:%SZ') > cutoff_date]

    #Get the quizzes lock_at times
    #quizzes_utimes_all = [f_.lock_at for f_ in list_quizzes if f_.lock_at != None ]
    #Get the quizzes due times
    #quizzes_ctimes_all = [f_.due_at for f_ in list_quizzes if f_.due_at != None  ]
    
    quizzes_utimes = []
    quizzes_ctimes = []
    for f_ in list_quizzes:
        try:
            #if datetime.strptime(f_.lock_at, '%Y-%m-%dT%H:%M:%SZ') > cutoff_date:
            if f_.lock_at != None:
                quizzes_utimes.append(f_.lock_at)
        except:
            pass
        try:
            if f_.due_at != None:
                quizzes_ctimes.append(f_.due_at)
        except:
            pass                

    
    #Get the discussion topics last reply times
    dtopics_utimes_all = [f_.last_reply_at for f_ in list_dis_topics if f_.last_reply_at != None ]        
    dtopics_utimes = []
    for f_ in list_dis_topics:
        try:
            if datetime.strptime(f_.last_reply_at, '%Y-%m-%dT%H:%M:%SZ') > cutoff_date:
                dtopics_utimes.append(f_.last_reply_at)
        except:
            pass
        

    #Get the discussion topics created times
    dtopics_ctimes_all = [f_.created_at for f_ in list_dis_topics ]
    dtopics_ctimes = [f_.created_at for f_ in list_dis_topics if datetime.strptime(f_.created_at, 
                                                                    '%Y-%m-%dT%H:%M:%SZ') > cutoff_date]    
    
    
    fa_times_all = file_utimes_all + assn_utimes_all + page_utimes_all + dtopics_utimes_all
    fa_c_times_all = file_ctimes_all + assn_ctimes_all + page_ctimes_all + dtopics_ctimes_all

    fa_times = file_utimes + assn_utimes + page_utimes + dtopics_utimes
    fa_c_times = file_ctimes + assn_ctimes + page_ctimes + dtopics_ctimes


    #Convert the whole thing 
    fa_times_all = np.array(fa_times_all, dtype='datetime64')
    fa_c_times_all = np.array(fa_c_times_all, dtype='datetime64')

    fa_times = np.array(fa_times, dtype='datetime64')
    fa_c_times = np.array(fa_c_times, dtype='datetime64')      


    try:
        fa_max = fa_times.max() if len(fa_times)>0 else fa_times_all.max()
    except:
        fa_max = None
        
    try:
        fa_c_min = fa_c_times.min() if len(fa_c_times)>0 else fa_c_times_all.min()
    except:
        fa_c_min = None

    #find the farthest due/lock_at date for quizzes:
    q_times = quizzes_utimes + quizzes_ctimes
    q_times = np.array(q_times, dtype='datetime64')

    try:
        q_max = q_times.max() if len(q_times)>0 else None
    except:
        q_max = None
  
    return [course_id, course_sis_id, course_dept, enrollment_term, parent_account, course_name, course_state,
            len(file_utimes_all), len(assn_utimes_all), len(fa_times_all), 
            fa_max, fa_c_min, num_students, is_public, public_syllabus,
           public_syllabus_to_auth, is_public_to_auth_users, course_visibility, num_published_dis_topics,
            num_published_pages, num_published_quizzes, num_assignment_groups,num_modules, num_module_items, q_max]



In [9]:
#Get a list of all department names by the sub-account id:
acc = canvas.get_account(1)
sub_account_dict = {}
sub_account_dict[1] = 'MIT Root Account'
parent_account_dict = {}
accs = acc.get_subaccounts(recursive=True)
for a_ in accs:
    sub_account_dict[a_.id] = a_.name

for a_ in accs:
    try:
        parent_account_dict[a_.id] = sub_account_dict[a_.parent_account_id]
    except Exception as e:
        pass

    
#Get course terms associated with the account and create a dictionary of the term names:
term_dict ={}
for term_ in acc.get_enrollment_terms():
    term_dict[term_.id] = term_.name
    
#parent_account_dict
#sub_account_dict

In [10]:
#Get a list of all courses:
large_num = 100000 #give a very large number so all courses are pulled recursively
#excl_acc = [1,17,76] #mention sub-accounts to be excluded
excl_acc = [] #Don't implement it here...
begin_date = '2015-05-01' #Date since when we are counting

course_rows = []
course_cols = ['course_id', 'course_code', 'course_name', 'account_id', 'created_at']
for acc_course in acc.get_courses()[:large_num]:
    course_rows.append([acc_course.id, acc_course.course_code, acc_course.name, acc_course.account_id, 
                        acc_course.created_at])
    
courses_df = pd.DataFrame.from_records(course_rows, columns=course_cols)

courses_df_dep_excluded = courses_df[~courses_df.account_id.isin(excl_acc)]
'''
try:
    courses_df_dep_excluded['Dept'] = courses_df_dep_excluded.apply(lambda 
                                                        row: sub_account_dict[row['account_id']], axis=1)
except:
    pass
courses_df_dep_excluded
'''
filtered_courses_df = courses_df_dep_excluded[courses_df_dep_excluded.created_at>=begin_date]

filtered_courses_df.to_csv('all_canvas_since_{}.csv'.format(begin_date), index=None)
filtered_courses_df

,course_id,course_code,course_name,account_id,created_at
0,1,welcome-to-canvas,Welcome to Canvas,1,2016-01-07T16:30:12Z
1,16,GSL,GSL Canvas Pilot,15,2016-10-03T14:39:27Z
2,17,21G.067/21G.597,Digital Media in Japan and Korea,11,2016-10-03T19:42:20Z
3,18,21G.614,Russian IV,14,2016-10-11T13:39:46Z
4,20,21G.501,Japanese I (Section 1),11,2016-10-11T13:55:59Z
...,...,...,...,...,...
15480,17561,crawfjen:20220824,Temporary Sandbox,139,2022-08-24T06:03:56Z
15481,17562,6.4212,6.4212 Robotic Manipulation,96,2022-08-24T15:19:40Z
15482,17563,LING123,Industry workshop 2022,139,2022-08-26T12:02:55Z
15483,17564,IDS.330,IDS.330 Real Optns for Prod & Sys Dsgn,106,2022-08-27T12:33:25Z


In [11]:
#Check that the migration/production/dev is set correctly
all_course_list_0 = filtered_courses_df.course_id.tolist()
all_course_list_0_rows = []
all_course_list_0_cols = ['course_id', 'course_sis_id', 'Dept', 'enrollment_term','parent_account', 'Course_name', 
    'course_state', 'num_files','num_assignments','num_tot_fa','last_update_at','first_created_at', 'num_students',
            'is_public', 'public_syllabus', 'public_syllabus_to_auth', 'is_public_to_auth_users', 'course_visibility',
            'num_published_dis_topics','num_published_pages', 'num_published_quizzes', 'num_assignment_groups',
                          'num_modules', 'num_module_items','farthest_quiz_due_lock_date']

if run_mode == 'dev' or run_mode=='mig':
    num_courses = 5
elif run_mode == 'prod' or run_mode=='prod_home':
    num_courses = len(all_course_list_0) + 1
    
for c_id in all_course_list_0[:num_courses]:
    all_course_list_0_rows.append(get_course_info(c_id))
    
all_course_list_0_df = pd.DataFrame.from_records(all_course_list_0_rows, columns=all_course_list_0_cols)
all_course_list_0_df['if_LT_led'] = 0
all_course_list_0_df['LT_email'] = np.nan
print(all_course_list_0_df.shape)
all_course_list_0_df.tail()


<ipython-input-8-84a9b5a8d51a>:232: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_times_all = np.array(fa_times_all, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:233: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_c_times_all = np.array(fa_c_times_all, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:235: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_times = np.array(fa_times, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:236: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_c_times = np.array(fa_c_times, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:251: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  q_times = np.array(q_times, dtype='datetime64')


(5, 27)


,course_id,course_sis_id,Dept,enrollment_term,parent_account,Course_name,course_state,num_files,num_assignments,num_tot_fa,...,course_visibility,num_published_dis_topics,num_published_pages,num_published_quizzes,num_assignment_groups,num_modules,num_module_items,farthest_quiz_due_lock_date,if_LT_led,LT_email
0,1,welcome-to-canvas,MIT Root Account,Default Term,None,Welcome to Canvas,unpublished,3,20,28,...,public,4,5,3,1,6,38,NaT,0,NaN
1,16,None,Manually-Created Courses,Default Term,MIT Root Account,GSL Canvas Pilot,available,0,1,2,...,not_public_to_auth_users,1,0,1,1,0,0,2016-10-08 03:59:00,0,NaN
2,17,21G.067/21G.597.2017SP,Japanese,Spring 2017,Global Studies and Languages,Digital Media in Japan and Korea,available,52,9,75,...,not_public_to_auth_users,12,2,0,1,14,64,NaT,0,NaN
3,18,21G.614.2017SP,Russian,Spring 2017,Global Studies and Languages,Russian IV,available,93,44,146,...,not_public_to_auth_users,0,8,15,1,6,53,2017-03-03 06:59:59,0,NaN
4,20,None,Japanese,IAP 2017,Global Studies and Languages,Japanese I (Section 1),available,2,6,11,...,institute,0,3,0,1,9,0,NaT,0,NaN


In [12]:
#Check that the migration/production/dev is set correctly
lt_courses_row = []
lt_courses_cols = all_course_list_0_cols + ['if_LT_led','LT_email']
courses_to_exclude = [3157, 3158]
lt_list = lt_df.Email.tolist()

if run_mode == 'dev' or run_mode=='mig':
    num_lts = 5
elif run_mode == 'prod' or run_mode=='prod_home':
    num_lts = len(lt_list) + 1


for user_email in lt_list[:num_lts]:
    try:
        user_ = canvas.get_user(user_email, 'sis_login_id')
        course_list = []
        user_courses = user_.get_enrollments(type=['TeacherEnrollment'])

        for uc_ in user_courses:
            #print(uc_.id, uc_.course_id)
            if uc_.course_id not in courses_to_exclude:
                uc_row = get_course_info(uc_.course_id)
                #print(uc_row)
                uc_row.extend([1, user_email])
                lt_courses_row.append(uc_row)
    except Exception as e:
        print('Error {} for user {}'.format(e, user_email))
        pass


all_lt_courses = pd.DataFrame.from_records(lt_courses_row, columns=lt_courses_cols)
print(all_lt_courses.shape)
#all_lt_courses


<ipython-input-8-84a9b5a8d51a>:232: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_times_all = np.array(fa_times_all, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:233: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_c_times_all = np.array(fa_c_times_all, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:235: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_times = np.array(fa_times, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:236: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  fa_c_times = np.array(fa_c_times, dtype='datetime64')
<ipython-input-8-84a9b5a8d51a>:251: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  q_times = np.array(q_times, dtype='datetime64')


(13, 27)


In [13]:
all_courses_df = pd.concat([all_course_list_0_df, all_lt_courses], ignore_index=True)
#all_courses_df

In [14]:
ts_cutoff_1w = pd.to_datetime('now') - pd.to_timedelta('7days')
ts_cutoff_1m = pd.to_datetime('now') - pd.to_timedelta('30days')
all_courses_df['if_active_last_week'] = np.where(all_courses_df.last_update_at>ts_cutoff_1w, True, False)
all_courses_df['if_active_last_month'] = np.where(all_courses_df.last_update_at>ts_cutoff_1m, True, False)
all_courses_df['if_active_since_June1'] = np.where(all_courses_df.first_created_at>=pd.to_datetime('2020-06-01'),
                                                   True, False)
all_courses_df['if_sandbox_course'] = np.where(all_courses_df.Dept=='Sandboxes', 1, 0)
#all_courses_df['if_default_fa'] = np.where(all_courses_df.num_tot_fa==26, 1, 0)
all_courses_df['if_default_fa'] = np.where(((all_courses_df.num_files==22) & 
                                            (all_courses_df.num_assignments==4)), 1, 0)

all_courses_df = all_courses_df[all_courses_df.course_id.notna()].reset_index(drop=True)
all_courses_df.drop_duplicates(subset=['course_id'], keep='last', inplace=True)
#all_courses_df.tail()

/Users/anindyaroy/opt/anaconda3/envs/venv/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:2224: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [15]:
all_courses_df['last_update_at'] = all_courses_df['last_update_at'].dt.strftime('%Y-%m-%d')
all_courses_df['first_created_at'] = all_courses_df['first_created_at'].dt.strftime('%Y-%m-%d')
all_courses_df['farthest_quiz_due_lock_date'] = all_courses_df['farthest_quiz_due_lock_date'].dt.strftime('%Y-%m-%d')
#all_courses_df.tail()

In [16]:
#Exclude the last day's information as that's still incomplete.
all_courses_df['first_created_at'] = pd.to_datetime(all_courses_df['first_created_at'])
all_courses_df = all_courses_df[all_courses_df.first_created_at!= all_courses_df['first_created_at'].max()]

all_courses_df.to_gbq('lt_courses.{}'.format(out_table), project_id, if_exists='replace', credentials=credentials)
#all_courses_df.to_csv('all_courses.csv', index=None)

### Find total courses by date snapshot

We'll take a snapshot of all the previous courses, and for each day, we'll add only the new numbers.